In [1]:
import numpy as np
import pywt

import seaborn as sns #绘制confusion matrix heatmap

import os
import scipy.io as sio

from statsmodels.tsa.ar_model import AR

import tqdm
import  time

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import warnings
warnings.simplefilter('ignore') #忽略警告

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.mixture import GaussianMixture

import xgboost

In [3]:
sample_rate = 256
origin_channel = 16


SAMPLE_CHANNEL = ['Pz' , 'PO3' , 'PO4' , 'O1' , 'O2' , 'Oz' , 'O9' , 'FP2' ,
                  'C4' , 'C6' , 'CP3' , 'CP1' ,
                  'CPZ' , 'CP2' , 'CP4' , 'PO8']

LABEL2STR = {0:'sen' , 1:'hong' , 2:'zhao',
             3:'fen' , 4:'xiao' , 5:'yu' , 
             6:'bin' , 7:'wang' , 8:'wei' , 
             9:'fei'}

CLIP_FORWARD = 1 #首部裁掉时间
CLIP_BACKWARD = 1 #尾部裁掉时间

trial_time = 3 #segment second


#是否进行归一化
#reference:a study on performance increasing in ssvep based bci application
#IS_NORMALIZE = True

#是否进行滤波
#IS_FILTER = False
#EEG频率范围
#reference:a study on performance increasing in ssvep based bci application
LO_FREQ = 0.5
HI_FREQ = 40

#是否陷波
#IS_NOTCH = False
NOTCH_FREQ = 50 #陷波 工频

from keras.utils import to_categorical

Using TensorFlow backend.


In [4]:
def load_data(filename):

    data = sio.loadmat(file_name=filename)['data_received'] #length*16 matrix

    data = data[CLIP_FORWARD * sample_rate : - CLIP_BACKWARD * sample_rate] #首部 尾部 进行裁剪
   
    return data 

In [5]:
def separate(data , label , overlap_length):
    train_data = []
    train_labels = []

    size = sample_rate * trial_time #一小段 256*3 个数据点
    data_length = data.shape[0]

    idx = 0
    
    while idx<=data_length-size:
        train_data.append(data[idx : idx+size , :])
        train_labels.append(label)

        idx = idx + (size - overlap_length)
        
    return np.array(train_data) , np.array(train_labels)

In [6]:
def shuffle_t_v(filenames):
    # np.random.shuffle(filenames)
    
    return np.random.choice(filenames , size=10) #20次的计算准确率中 每次随机选择10个样本进行训练测试

def combine(freq):    
    overlap_length = 2*256 #重叠2秒数据
    
    #保证随机性 进行置乱
    person_0_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/0/' % freq) )
    person_1_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/1/' % freq) )
    person_2_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/2/' % freq) )
    person_3_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/3/' % freq) )
    person_4_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/4/' % freq) )
    person_5_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/5/' % freq) )
    person_6_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/6/' % freq) )
    person_7_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/7/' % freq) )
    person_8_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/8/' % freq) )

    #打开信号文件 并 合并
    person_0 = np.concatenate([load_data('../incremental/data/base_rf/%s/0/' % freq + filename) for filename in person_0_filenames] , axis = 0)
    person_1 = np.concatenate([load_data('../incremental/data/base_rf/%s/1/' % freq + filename) for filename in person_1_filenames] , axis = 0)
    person_2 = np.concatenate([load_data('../incremental/data/base_rf/%s/2/' % freq + filename) for filename in person_2_filenames] , axis = 0)
    person_3 = np.concatenate([load_data('../incremental/data/base_rf/%s/3/' % freq + filename) for filename in person_3_filenames] , axis = 0)
    person_4 = np.concatenate([load_data('../incremental/data/base_rf/%s/4/' % freq + filename) for filename in person_4_filenames] , axis = 0)
    person_5 = np.concatenate([load_data('../incremental/data/base_rf/%s/5/' % freq + filename) for filename in person_5_filenames] , axis = 0)
    person_6 = np.concatenate([load_data('../incremental/data/base_rf/%s/6/' % freq + filename) for filename in person_6_filenames] , axis = 0)
    person_7 = np.concatenate([load_data('../incremental/data/base_rf/%s/7/' % freq + filename) for filename in person_7_filenames] , axis = 0)
    person_8 = np.concatenate([load_data('../incremental/data/base_rf/%s/8/' % freq + filename) for filename in person_8_filenames] , axis = 0)
    
    #============
    #训练数据分段
    train_person_data_0 , train_person_labels_0 = separate(person_0 , label = 0 , overlap_length=overlap_length)
    train_person_data_1 , train_person_labels_1 = separate(person_1 , label = 1 , overlap_length=overlap_length)
    train_person_data_2 , train_person_labels_2 = separate(person_2 , label = 2 , overlap_length=overlap_length)
    train_person_data_3 , train_person_labels_3 = separate(person_3 , label = 3 , overlap_length=overlap_length)
    train_person_data_4 , train_person_labels_4 = separate(person_4 , label = 4 , overlap_length=overlap_length)
    train_person_data_5 , train_person_labels_5 = separate(person_5 , label = 5 , overlap_length=overlap_length)
    train_person_data_6 , train_person_labels_6 = separate(person_6 , label = 6 , overlap_length=overlap_length)
    train_person_data_7 , train_person_labels_7 = separate(person_7 , label = 7 , overlap_length=overlap_length)
    train_person_data_8 , train_person_labels_8 = separate(person_8 , label = 8 , overlap_length=overlap_length)

    #合并数据
    train_data = np.concatenate((train_person_data_0 , train_person_data_1 , train_person_data_2 ,
                                 train_person_data_3 , train_person_data_4 , train_person_data_5 ,
                                 train_person_data_6 , train_person_data_7 , train_person_data_8 ,
                                 ))
    
    train_labels = np.concatenate((train_person_labels_0 , train_person_labels_1 , train_person_labels_2 ,
                                   train_person_labels_3 , train_person_labels_4 , train_person_labels_5 ,
                                   train_person_labels_6 , train_person_labels_7 , train_person_labels_8 ,
                                    ))
    
    #产生索引并置乱
    idx_train_data = list(range(train_data.shape[0]))
    np.random.shuffle(idx_train_data)

    #将训练数据置乱
    train_data = train_data[idx_train_data]
    train_labels = train_labels[idx_train_data]
        
    return train_data , train_labels


In [7]:
def session_data_labels(session_id , freq , is_training):
    if is_training:
        overlap_length = 256*2
    else:
        overlap_length = 0
    
    str_freq = str(freq)
    
    subjcets = os.listdir('../incremental/data/incremental/%s/s%d/' % (str_freq , session_id)) #受试者ID
    
    data = []
    labels = []
    
    for subjcet in subjcets:
        filenames = os.listdir('../incremental/data/incremental/%s/s%d/%s/' % (str_freq , session_id , subjcet))
        
        person = np.concatenate([load_data('../incremental/data/incremental/%s/s%d/%s/%s' % (str_freq , session_id , subjcet , filename)) for filename in filenames] , axis = 0)
        
        person_data , person_label = separate( person , label = int(subjcet) , overlap_length = overlap_length)
        
        data.append(person_data)
        labels.append(person_label)
    
    #合并数据
    data = np.concatenate(data)
    labels = np.concatenate(labels)
    
    #shuffle
    idx_data = list(range(data.shape[0]))
    np.random.shuffle(idx_data)

    data = data[idx_data]
    labels = labels[idx_data]
    
    return data , labels


# PCA降维
## kmeans聚类

In [8]:
from keras.models import Sequential , Model
from keras.layers import Dense , Dropout , Conv2D , MaxPooling2D , Reshape , BatchNormalization , Flatten
from keras.layers import Input , LSTM

In [9]:
encoder_input = Input(shape = (256*3 , 16))

#encoder
encoder = LSTM(32 , return_sequences=True)(encoder_input)
encoder_output = Dense(64 , activation='relu')(encoder)
encoder_output = Flatten()(encoder_output)
encoder_output = Dense(64 , activation='relu')(encoder_output)

#decoder
decoder = Dense(768*64 , activation='relu')(encoder_output)
decoder = Reshape(target_shape=(768 , 64))(decoder)
decoder = LSTM(32)(decoder)
decoder_output = Dense(256*3*16 , activation='relu')(decoder)
decoder_output = Reshape(target_shape=(256*3 , 16))(decoder_output)

In [10]:
autoencoder = Model(inputs=encoder_input , outputs=decoder_output)
encoder = Model(inputs=encoder_input , outputs=encoder_output)

## 注意 分两次运行 6 7.5 和 8.5 10

In [13]:
for freq in [6 , 7.5]:
    
    print('freq : ' , freq)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #自编码器的训练
        autoencoder.compile(optimizer='adam' , loss = 'mse')
        autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0)
        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
        
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

freq :  6
session :  1 [[-103.30456   -42.774944]
 [ 990.41473     6.174515]
 [-145.89462   493.09192 ]
 [-140.4229   -111.534676]
 [-139.74927   597.19586 ]
 [-147.90546   346.20505 ]
 [-150.65353    63.878716]
 [-156.25842   -23.859875]
 [-144.73189   -48.534504]] 0.096361
session :  3 [[-1.4558879e+02 -1.8367999e+00]
 [ 1.1654363e+03  2.5297263e-01]
 [-1.4851292e+02  4.1702633e+01]
 [-1.4292363e+02 -3.9863430e+01]
 [-1.4500789e+02 -1.0390284e+01]
 [-1.4408551e+02 -2.3640768e+01]
 [-1.4617097e+02  7.2626395e+00]
 [-1.4800337e+02  3.4528839e+01]
 [-1.4532034e+02 -5.9245410e+00]] 0.05057059999944613
session :  5 [[ 195.77013   -33.556423]
 [-101.32226   -24.268118]
 [  29.592646  140.68901 ]
 [ 167.79861    53.64836 ]
 [  38.217876   65.95049 ]
 [ 172.45627   -67.90003 ]
 [ -34.919937   97.60494 ]
 [ 247.20178    73.007256]
 [-108.37393    15.763445]] 0.09629039999890665
session :  6 [[-59.345722  -55.152966 ]
 [495.9683      5.7400436]
 [-47.148582  135.40564  ]
 [-92.9052    262.7744

ResourceExhaustedError: OOM when allocating tensor with shape[32,768,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: lstm_2/zeros_like = ZerosLike[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](reshape_1/Reshape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: loss_25/mul/_3577 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3501_loss_25/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [12]:
for freq in [8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
        
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

freq :  8.5
session :  1 [[ -31.723572  -42.898712]
 [ 350.3053    155.9407  ]
 [-204.6263    269.1208  ]
 [   7.566886  -80.769295]
 [ -58.10243   -59.060368]
 [  35.45666   -97.25578 ]
 [ -30.031178  -64.00862 ]
 [-221.98976   308.12787 ]
 [  -4.66879   -13.238199]] 0.08240399999885994
session :  3 [[-63.422047  -72.85427  ]
 [342.97824    11.071364 ]
 [-71.45408   203.32275  ]
 [-30.139849  -26.565655 ]
 [-42.127884    3.7717018]
 [-38.698097  -45.06139  ]
 [382.1682     14.753518 ]
 [-30.86456    28.599566 ]
 [-40.086662  -31.166267 ]] 0.05486860000019078
session :  5 [[ 165.77715    139.71977  ]
 [   9.865319   -34.81572  ]
 [ -67.51116     10.893671 ]
 [  54.349102   -84.14891  ]
 [-145.87852     77.277824 ]
 [  41.775963   -30.38363  ]
 [ -23.136066    -3.6104004]
 [  79.18785    -68.01758  ]
 [  42.559483  -106.16223  ]] 0.056353200001467485
session :  6 [[ 313.23605    245.0128   ]
 [-131.49294     40.055923 ]
 [  87.609085  -147.69553  ]
 [ 175.73784   -168.79117  ]
 [-202.39

## DBSCAN聚类

In [11]:
for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        dbscan = DBSCAN()
        start = time.clock()
        res = dbscan.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)

freq :  6
session :  1 [[-138.7129    297.2712  ]
 [1310.5975    -93.60619 ]
 [-250.63196  -272.39484 ]
 [-169.59291   -44.553272]
 [ -58.229546  186.34444 ]
 [-185.2595    478.83725 ]
 [  67.37501   263.4952  ]
 [-121.95158    78.429146]
 [-209.90662  -171.76633 ]] 0.059514700000000004
session :  3 [[-1.8803296e+02  8.9529400e+00]
 [ 1.5078998e+03 -2.0213323e+00]
 [-2.0117903e+02 -4.8392544e+01]
 [-1.6836658e+02  7.1393005e+01]
 [-1.7869119e+02  3.2342598e+01]
 [-1.9141826e+02 -7.3650522e+00]
 [-2.0328111e+02 -6.5394836e+01]
 [-1.8588985e+02  1.8041756e+01]
 [-1.8926936e+02  6.0011333e-01]] 0.04860399999999743
session :  5 [[-338.1623   -167.72446 ]
 [ 722.29926   -47.108364]
 [-250.8307    214.95482 ]
 [ -14.868722  175.67842 ]
 [-307.87552  -121.93938 ]
 [ 513.952      21.243168]
 [-178.19682   363.8366  ]
 [ 630.1675    -16.565285]
 [-276.96088   -66.96561 ]] 0.054015800000001946
session :  6 [[1956.3553     33.219997]
 [-238.5462   -152.66092 ]
 [-326.80298  1084.9767  ]
 [-388.49

session :  13 [[ -708.7093    -232.54137 ]
 [ 1688.3551   -1077.4382  ]
 [  925.8351    1491.0358  ]
 [ -215.08488    -48.08603 ]
 [   19.80037    314.6726  ]
 [ -406.39957   -117.592545]
 [ -134.5351     136.1686  ]
 [  101.81758    411.3189  ]
 [ -327.65445    -85.18546 ]] 0.053002599997853395
freq :  10
session :  1 [[-436.2955     60.680683]
 [1230.6445    821.02124 ]
 [ 912.3335   -850.08527 ]
 [  45.79504  -469.46237 ]
 [ 116.70336   225.19972 ]
 [-689.6668    115.36678 ]
 [-116.993355  -98.01127 ]
 [-388.26395   139.173   ]
 [ 109.4816   -585.2302  ]] 0.053358599994680844
session :  3 [[-623.27313  -318.4388  ]
 [3542.4705   -276.71674 ]
 [  24.309032  934.30554 ]
 [-483.742    -150.21236 ]
 [-652.4061   -387.99097 ]
 [-522.5905    -94.8728  ]
 [ -12.143808  973.49304 ]
 [ -17.647076  881.2186  ]
 [-570.4142   -204.91429 ]] 0.05186659999890253
session :  5 [[ 345.22232   748.5816  ]
 [-290.55136  -535.7936  ]
 [1169.4149   -657.02515 ]
 [-315.8828   -217.58473 ]
 [  10.775166  3

## GMM聚类

In [ ]:
for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)
    

freq :  6


# LDA降维
## kmeans聚类

In [ ]:
for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_lda = lda.fit_transform(X_sbp , y)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
        
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

## DBSCAN聚类

In [ ]:
for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_lda = lda.fit_transform(X_sbp , y)
        time1 = time.clock() - start
        
        dbscan = DBSCAN()
        start = time.clock()
        res = dbscan.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)
    

In [ ]:
for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_lda = lda.fit_transform(X_sbp , y)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9 , tol=0.001)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
        
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

## GMM聚类

In [ ]:
for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_lda = lda.fit_transform(X_sbp , y)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)
    